In [20]:
import urllib.request
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [105]:
import dgl

ratings = dgl.heterograph(
    {('user', 'rating', 'menuItem') : [(0, 0), (0, 1), (1, 0)],
     ('menuItem', 'contains', 'ingredient') : [(2, 1)]})
ratings.edges['rating'].data['val'] = torch.tensor([2,4,5])
ratings.nodes['user'].data['I'] = torch.tensor([1,1])
ratings.nodes['menuItem'].data['I'] = torch.tensor([1,1,1])

In [106]:
print(ratings.num_nodes())
print('Node types:', ratings.ntypes)
print('Edge types:', ratings.etypes)
print('Canonical edge types:', ratings.canonical_etypes)
print(ratings.edata)
print(ratings.ndata)

7
Node types: ['ingredient', 'menuItem', 'user']
Edge types: ['contains', 'rating']
Canonical edge types: [('menuItem', 'contains', 'ingredient'), ('user', 'rating', 'menuItem')]
defaultdict(<class 'dict'>, {'val': {('user', 'rating', 'menuItem'): tensor([2, 4, 5])}})
defaultdict(<class 'dict'>, {'I': {'menuItem': tensor([1, 1, 1]), 'user': tensor([1, 1])}})


In [107]:
ratings.edges(etype='rating')
for c_etype in ratings.canonical_etypes:
    srctype, etype, dsttype = c_etype
    print(c_etype)
    print(ratings.edges[etype])

('menuItem', 'contains', 'ingredient')
EdgeSpace(data={})
('user', 'rating', 'menuItem')
EdgeSpace(data={'val': tensor([2, 4, 5])})


In [108]:
import pygraphviz as pgv
def plot_graph(nxg):
    ag = pgv.AGraph(strict=False, directed=True)
    for u, v, k in nxg.edges(keys=True):
        ag.add_edge(u, v, label=k)
    ag.layout('dot')
    ag.draw('graph.png')
 
print(ratings.metagraph().edges())
plot_graph(ratings.metagraph())

[('menuItem', 'ingredient'), ('user', 'menuItem')]


In [109]:
class MLPPredictor(nn.Module):
    def __init__(self, in_features, out_classes):
        super().__init__()
        self.W = nn.Linear(in_features * 2, out_classes)

    def apply_edges(self, edges):
        
        h_u = edges.src['h']
        h_v = edges.dst['h']
        score = self.W(torch.cat([h_u, h_v], 1))
        return {'score': score}

    def forward(self, graph, h, etype):
        # h contains the node representations computed from the GNN defined
        # in the node classification section (Section 5.1).
        with graph.local_scope():
            graph.ndata['h'] = h
            graph.apply_edges(self.apply_edges, etype=etype)
            return graph.edata['score']

In [110]:
class HeteroDotProductPredictor(nn.Module):
    def forward(self, graph, etype):
        print("Here")
        with graph.local_scope():
            graph.apply_edges(fn.u_mul_e('I', 'val', 'score'), etype=etype)
            return graph.edges[etype].data['score']

In [111]:
class RGCN(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, rel_names):
        super().__init__()

        self.conv1 = dgl.nn.HeteroGraphConv({
            rel: dgl.nn.GraphConv(in_feats, hid_feats)
            for rel in rel_names}, aggregate='sum')
        self.conv2 = dgl.nn.HeteroGraphConv({
            rel: dgl.nn.GraphConv(hid_feats, out_feats)
            for rel in rel_names}, aggregate='sum')

    def forward(self, graph, inputs):
        # inputs are features of nodes
        h = self.conv1(graph, inputs)
        h = {k: F.relu(v) for k, v in h.items()}
        h = self.conv2(graph, h)
        return h

In [112]:
class Model(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, rel_names):
        super().__init__()
        self.sage = RGCN(in_features, hidden_features, out_features, rel_names)
        self.pred = HeteroDotProductPredictor()
    def forward(self, g, x, etype):
        print(g, x, etype)
        h = self.sage(g, x)
        return self.pred(g, h, etype)

In [125]:
model = Model(2, 20, 5, ratings.etypes)
user_feats = ratings.nodes['user'].data['I']
item_feats = ratings.nodes['menuItem'].data['I']
##label = ratings.edges['click'].data['label']
#train_mask = ratings.edges['click'].data['train_mask']
node_features = {'user': user_feats, 'menuItem': item_feats, 'ingredient': torch.tensor([1])}

In [126]:
opt = torch.optim.Adam(model.parameters())
for epoch in range(10):
    pred = model(ratings, node_features, 'rating')
    loss = ((pred - label) ** 2).mean()
    opt.zero_grad()
    loss.backward()
    opt.step()
    print(loss.item())

Graph(num_nodes={'ingredient': 2, 'menuItem': 3, 'user': 2},
      num_edges={('menuItem', 'contains', 'ingredient'): 1, ('user', 'rating', 'menuItem'): 3},
      metagraph=[('menuItem', 'ingredient', 'contains'), ('user', 'menuItem', 'rating')]) {'user': tensor([1, 1]), 'menuItem': tensor([1, 1, 1]), 'ingredient': tensor([1])} rating


RuntimeError: The size of tensor a (20) must match the size of tensor b (2) at non-singleton dimension 0